# Time-based operations

In [ ]:
import subprocess as sp

In [ ]:
# get the plot in notebook
%matplotlib inline

In [ ]:
import xarray as xr

In [ ]:
# set up where the run's postprocessing live
archdir = '/archive/Raphael.Dussin/xanadu_esm4_20190304_mom6_2019.08.08/'
storedir = archdir + 'OM4p25_JRA55do1.4_0netfw_cycle6/gfdl.ncrc4-intel16-prod/pp/'

## 1. Computing climatologies for SST

In [ ]:
# open the Potential temperature array store
datadir = storedir + 'ocean_monthly_z/ts/monthly/5yr/'
variable = 'thetao'
# at GFDL, we need to get data off tapes
check = sp.check_call(f'dmget -v -d {datadir} *{variable}.nc', shell=True)

In [ ]:
# open the data store
ds = xr.open_mfdataset(f'{datadir}/*.{variable}.nc', combine='by_coords')

In [ ]:
# inspect the dataset
ds

As a first test, let's compute the time-average of the full time serie. To get things running fast, we set up a dask cluster.

In [ ]:
# import the client, this is going to be our interface to see what happens on the DASK cluster
from dask.distributed import Client

In [ ]:
# SLURMCluster will allow us to submit a job to the SLURM batch scheduler
# that will run the DASK cluster
from dask_jobqueue import SLURMCluster

In [ ]:
# Set up SLURM options
cluster = SLURMCluster(queue='analysis', cores=8, project='gfdl_o', memory="96GB", walltime="1:00:00")
# submit the job for N=1 nodes (this is the number of workers, not cores)
cluster.scale(1)
# connect the client side to cluster
client = Client(cluster)

In [ ]:
client

### 1.1 Annual climatology

In [ ]:
climato_sst = ds['thetao'].sel(z_l=2.5).mean(dim='time')

In [ ]:
climato_sst

The result is instaneous because none of the computation actually happened. This is called lazy evaluation, xarray only returns the structure of the resulting data array. Calling plot will explicitly trigger compute:

In [ ]:
climato_sst.plot(vmin=-2, vmax=38, cmap='gist_ncar')

Now go check the dashboard!

### 1.2 Monthly climatology

Using groupby, we can group data by month and then average over the years, resulting in a monthly climatology:

In [ ]:
sst = ds['thetao'].sel(z_l=2.5)
monthly_clim_sst = sst.groupby(sst['time'].dt.month).mean(dim=['time'])

In [ ]:
monthly_clim_sst

Let's look at the climatological seasonal cycle in zonal average. For an irregular grid such as this one, the x-mean is not a correct zonal mean in the arctic. See tutorial on spatial averages for correct computation.

In [ ]:
monthly_clim_sst.mean(dim='xh').plot(vmin=-2, vmax=35, cmap='jet', x='month')

### 1.3 Anomaly to seasonal cycle

In [ ]:
anom = sst.groupby(sst['time'].dt.month) - monthly_clim_sst

In [ ]:
anom

In [ ]:
anom.mean(dim='xh').plot(vmin=-2, vmax=2, cmap='bwr', x='time')

It's a bit noisy but We can make it annual with a new groupby:

In [ ]:
yearly_anom = anom.groupby(anom['time'].dt.year).mean(dim='time')

In [ ]:
yearly_anom

In [ ]:
yearly_anom.mean(dim='xh').plot(vmin=-2, vmax=2, cmap='bwr', x='year')

## 2. Selecting based on dates

Dates in the dataset are interpreted as time objects, which makes it easy to select a range of dates once you are familiar with how to write the query. 

In [ ]:
ds['time']

For example, selecting a range of years (think time **slice**):

In [ ]:
my_selection = ds.sel(time=slice('1981', '1990'))

In [ ]:
my_selection

Or just a few months:

In [ ]:
my_selection2 = ds.sel(time=slice('1981-04', '1981-07'))

In [ ]:
my_selection2

In [ ]:
client.close()

cluster.close()